In [15]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
from datetime import datetime

# --- CONFIGURATION ---
ASSETS = {
    'EURUSD': 'EURUSD=X',
    'USDCHF': 'USDCHF=X',
    'GBPUSD': 'GBPUSD=X',
    'USDCAD': 'USDCAD=X',
    'BTCUSD': 'BTC-USD',
    'ETHUSD': 'ETH-USD',
    'XAUUSD': 'XAUUSD=X',
    'XAGUSD': 'XAGUSD=X',
    'SP500m': '^GSPC',
    'UK100': '^FTSE'
}

TIMEFRAME_SETTINGS = {
    '1WK': ('1wk', '10y'),
    'D1':  ('1d', '5y'),
    '4H':  ('4h', '2y'),
    '1H':  ('1h', '2y'),
    'M30': ('30m', '60d'),
    'M15': ('15m', '60d')
}


def flatten_yf_columns(df):
    """Removes multi-index levels from yfinance columns."""
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(1)
    return df


def detect_fvg(data, lookback_period=28, body_multiplier=2.5):
    """
    Core Logic:
    1. Middle candle body > avg body * multiplier
    2. Gap > 10% of middle candle body
    """
    fvg_list = [None] * len(data)

    # Need at least 3 candles to check a gap
    for i in range(2, len(data)):
        first_high = data['High'].iloc[i-2]
        first_low = data['Low'].iloc[i-2]
        mid_open = data['Open'].iloc[i-1]
        mid_close = data['Close'].iloc[i-1]
        third_low = data['Low'].iloc[i]
        third_high = data['High'].iloc[i]

        mid_body = abs(mid_close - mid_open)

        # Calculate rolling average body size
        start_idx = max(0, i - 1 - lookback_period)
        prev_bodies = abs(data['Close'].iloc[start_idx:i-1] -
                          data['Open'].iloc[start_idx:i-1])
        avg_body_size = prev_bodies.mean() if not prev_bodies.empty else 0.001

        min_gap = mid_body * 0.10

        # Bullish FVG
        if third_low > first_high:
            gap_size = third_low - first_high
            if (mid_body > avg_body_size * body_multiplier) and (gap_size > min_gap):
                fvg_list[i] = ('bullish', first_high, third_low, i)

        # Bearish FVG
        elif third_high < first_low:
            gap_size = first_low - third_high
            if (mid_body > avg_body_size * body_multiplier) and (gap_size > min_gap):
                fvg_list[i] = ('bearish', first_low, third_high, i)

    return fvg_list


def plot_fvg_chart(pair, timeframe, candle_limit=100):
    """
    Main function to fetch data and plot FVGs.
    :param pair: String key from ASSETS (e.g., 'EURUSD')
    :param timeframe: String key from TIMEFRAME_SETTINGS (e.g., '4H')
    :param candle_limit: Number of recent candles to display
    """
    if pair not in ASSETS or timeframe not in TIMEFRAME_SETTINGS:
        print(f"Invalid pair or timeframe. Options: {list(ASSETS.keys())}")
        return

    interval, period = TIMEFRAME_SETTINGS[timeframe]
    symbol = ASSETS[pair]

    print(f"Fetching {pair} ({timeframe}) for the last {period}...")

    df = yf.download(symbol, period=period, interval=interval,
                     auto_adjust=False, progress=False)

    if df.empty:
        print("Failed to fetch data.")
        return

    df = flatten_yf_columns(df)
    df.reset_index(inplace=True)

    # Run Detection
    df['FVG'] = detect_fvg(df)

    # Slice for plotting
    df_plot = df.tail(candle_limit).copy()

    # Create Chart
    fig = go.Figure()

    # Candlesticks
    fig.add_trace(go.Candlestick(
        x=df_plot.index,
        open=df_plot["Open"],
        high=df_plot["High"],
        low=df_plot["Low"],
        close=df_plot["Close"],
        name=pair
    ))

    # Add Shapes
    for idx, row in df_plot.iterrows():
        if row['FVG'] is not None:
            fvg_type, start, end, original_idx = row['FVG']
            color = "rgba(0, 255, 100, 0.3)" if fvg_type == "bullish" else "rgba(255, 50, 50, 0.3)"

            fig.add_shape(
                type="rect",
                x0=idx - 1,  # Center on the middle candle
                x1=idx + 20,  # Extend forward
                y0=start,
                y1=end,
                fillcolor=color,
                layer="below",
                line=dict(width=0),
            )

    fig.update_layout(
        title=f"{pair} {timeframe} - Fair Value Gaps (Strict Filter)",
        template="plotly_dark",
        xaxis_rangeslider_visible=False,
        width=1200,
        height=700
    )

    fig.show()




In [16]:
# --- EXECUTION ---
if __name__ == "__main__":
    # You can call this with any pair and timeframe combination defined above
    plot_fvg_chart('EURUSD', '1H', candle_limit=150)
    # plot_fvg_chart('BTCUSD', '1H', candle_limit=100)

Fetching EURUSD (1H) for the last 2y...
